In [1]:
import json
from sklearn.cluster import OPTICS
import numpy as np

import tensorflow as tf
from tensorflow import keras

from sklearn.linear_model import SGDClassifier

In [2]:
with open('SmiteData-master/conquest_match_data/1649.json', 'r') as infile:
    raw_data = json.loads(infile.readline())

In [3]:
npdata = np.array([np.divide(np.array([x['assists'], x['damage_mitigated'], x['damage_player'], x['damage_taken'], x['deaths'], x['healing'], x['healing_player_self'], x['kills_player'], x['structure_damage']]), x['match_time_minutes']) for x in raw_data])
npdata_normalized = npdata / npdata.max(axis=0)
npdata_winlabel = np.array([1 if x['win_status'] == 'Winner' else 0 for x in raw_data])

In [4]:
npdata_normalized[0:5,:]

array([[0.17708333, 0.26098737, 0.24186276, 0.51195801, 0.36111111,
        0.        , 0.15863293, 0.14583333, 0.18933682],
       [0.17      , 0.21374689, 0.15833964, 0.45531513, 0.26      ,
        0.        , 0.22175235, 0.105     , 0.04463657],
       [0.19675926, 0.235752  , 0.54406125, 0.59636921, 0.18055556,
        0.        , 0.36874026, 0.4212963 , 0.27646527],
       [0.30357143, 0.39387455, 0.54629871, 0.63480488, 0.37142857,
        0.        , 0.26607378, 0.3       , 0.26662741],
       [0.20833333, 0.3522963 , 0.3798932 , 0.61548226, 0.38235294,
        0.        , 0.24534048, 0.20588235, 0.        ]])

In [5]:
npdata_normalized.shape

(4646, 9)

In [6]:
clustering = OPTICS(min_cluster_size=.2, max_eps=50).fit(npdata)

In [7]:
len([x for x in clustering.labels_ if x != -1])

1304

In [8]:
model = keras.Sequential([
    keras.layers.Dense(9),
    keras.layers.Dense(1)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

In [9]:
model.fit(npdata_normalized[:3000], npdata_winlabel[:3000], epochs=10)

Train on 3000 samples
Epoch 1/10
3000/3000 [==============================] - 2s 667us/sample - loss: 0.3900 - accuracy: 0.3947
Epoch 2/10
3000/3000 [==============================] - 0s 71us/sample - loss: 0.2698 - accuracy: 0.5373
Epoch 3/10
3000/3000 [==============================] - 0s 68us/sample - loss: 0.2113 - accuracy: 0.6803
Epoch 4/10
3000/3000 [==============================] - 0s 68us/sample - loss: 0.1735 - accuracy: 0.7663
Epoch 5/10
3000/3000 [==============================] - 0s 60us/sample - loss: 0.1513 - accuracy: 0.7973
Epoch 6/10
3000/3000 [==============================] - 0s 80us/sample - loss: 0.1397 - accuracy: 0.8133
Epoch 7/10
3000/3000 [==============================] - 0s 82us/sample - loss: 0.1341 - accuracy: 0.8263
Epoch 8/10
3000/3000 [==============================] - 0s 94us/sample - loss: 0.1306 - accuracy: 0.8360
Epoch 9/10
3000/3000 [==============================] - 0s 107us/sample - loss: 0.1284 - accuracy: 0.8380
Epoch 10/10
3000/3000 [========

In [11]:
model.evaluate(npdata_normalized[3000:], npdata_winlabel[3000:], verbose=2)

1646/1 - 0s - loss: 0.1097 - accuracy: 0.8433


[0.13246235608113058, 0.84325635]

In [10]:
classifier = SGDClassifier(max_iter=1000)
classifier.fit(npdata_normalized[:3000], npdata_winlabel[:3000])

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [12]:
classifier.score(npdata_normalized[3000:], npdata_winlabel[3000:])

0.8262454434993924